## Baseline model 

This model is a simple Linear regression with parameter of current RV for the target of future RV. 

In [1]:
import sys, importlib
import torch 
import torch.nn as nn
import numpy as np
import pandas as pd
import copy
import time

sys.path.append("../")
from proj_mod import training, data_processing
importlib.reload(training);
importlib.reload(data_processing);

# First build an approximate RV feature
path_book="../raw_data/kaggle_ORVP/book_train.parquet"
df_rv=data_processing.create_df_RV_by_row_id_parallel(path_book)

df_target=pd.read_csv("../raw_data/kaggle_ORVP/train.csv")
df_target["row_id"]=df_target["stock_id"].astype(int).astype(str)+"-"+df_target["time_id"].astype(int).astype(str)

list_time=np.load("../processed_data/recovered_time_id_order.npy")

time_split_list=data_processing.time_cross_val_split(list_time=list_time,n_split=1,percent_val_size=10,list_output=True)
train_time_id,test_time_id=time_split_list[0][0],time_split_list[0][1]

# Then merge with target dataframe
df_rv_target=pd.merge(df_target, df_rv, on=["row_id","time_id","stock_id"])

train_dataset_base = df_rv_target.loc[df_rv_target['time_id'].isin(train_time_id), ['RV','target','row_id']].set_index('row_id')
test_dataset_base = df_rv_target.loc[df_rv_target['time_id'].isin(test_time_id), ['RV','target','row_id']].set_index('row_id')

In fold 0 :

Train set end at 8117 .

Test set start at 15516 end at 10890 .



In [2]:
from sklearn.linear_model import LinearRegression

X_train=train_dataset_base[['RV']]
y_train=train_dataset_base['target']

X_test=test_dataset_base[['RV']]
y_test=test_dataset_base['target']

model= LinearRegression()
model.fit(X_train,y_train)

training_loss = training.rmspe(model.predict(X_train),y_train)
validation_loss = training.rmspe(model.predict(X_test),y_test)

print("The losses for training and validation data are: {:.4f} and {:.4f}".format(training_loss, validation_loss))

The losses for training and validation data are: 0.3019 and 0.2678
